# Loading Data to PostgreSQL server

### Import Libraries

In [9]:
import pandas as pd
import numpy as np
import time

# For loading data to postgres server
from odo import odo

# For connecting to postgres server
from sqlalchemy import create_engine

# For loading secret environment variables, e.g. postgres username and password
import os
from dotenv import load_dotenv, find_dotenv


### Load Environment Variables
**Instructions:**  
Store your SQL environment variables in a file called `.env` and save it in the root directory of this project.

Example: 

```
PUBLIC_IP = 12.345.678.901
USERNAME  = postgres
PASSWORD  = password
PORT      = 5432
```

In [2]:
# Find .env
dotenv_path = find_dotenv()

# Load entries as environment variables
load_dotenv(dotenv_path)

# Get secret environment variables 
public_ip = os.environ.get("PUBLIC_IP")
username  = os.environ.get("USERNAME")
password  = os.environ.get("PASSWORD")
port      = os.environ.get("PORT")

# Construct database URL from environment variables
uri = f'postgresql://{username}:{password}@{public_ip}:{port}'

### Load Data

In [12]:
chunksize = 1000

# review_chunks = pd.read_csv('../data/raw/yelp_academic_dataset_review.csv', chunksize = chunksize)
mon_ami_gabi = pd.read_csv('../data/interim/mon_ami_gabi.csv')
top_businesses = pd.read_csv('../data/interim/top_businesses.csv')

### Load data to Postgres on AWS

In [9]:
t0 = time.time()
odo(mon_ami_gabi, uri+'::mon_ami_gabi_reviews')
t1 = np.round(time.time() - t0, 2)

print(f'Loaded {mon_ami_gabi.shape[0]} rows in {t1} seconds')


Loaded 7968 rows in 258.8 seconds


In [13]:
t0 = time.time()
odo(top_businesses, uri+'::businesses')
t1 = np.round(time.time() - t0, 2)

print(f'Loaded {top_businesses.shape[0]} rows in {t1} seconds')


Loaded 1000 rows in 28.2 seconds


### Load data to Postgres in chunks

In [16]:
def next_chunk(chunks, n):
    '''
    Iterates through n chunks
    '''
    # Go to n'th chunk
    for i in range(n):
        chunk = next(chunks)
    return chunk

In [13]:
chunk = next_chunk(review_chunks, 10)

In [15]:
for i in range(10):
    t0 = time.time()
    chunk = next(review_chunks)
    odo(chunk, uri+'::reviews')
    t1 = np.round(time.time() - t0, 3))
    
    print(f'Loaded {chunk.index[0]} to {chunk.index[-1]} in {t1} seconds')

print('Run me again!')

32.702
Loaded 11000 to 11999
28.539
Loaded 12000 to 12999
33.829
Loaded 13000 to 13999
29.953
Loaded 14000 to 14999
33.466
Loaded 15000 to 15999
30.561
Loaded 16000 to 16999
35.261
Loaded 17000 to 17999
30.768
Loaded 18000 to 18999
34.158
Loaded 19000 to 19999
30.173
Loaded 20000 to 20999
Run me again!


This is too slow so I will load my data to a local Postgres server.

### Load data to local Postgres server

#### Set Environment Variables

In [5]:
raw_data_directory         = os.path.join('..', 'data', 'raw')
interim_data_directory     = os.path.join('..', 'data', 'interim')

review_filepath            = os.path.join(raw_data_directory, 'yelp_academic_dataset_review.csv')
business_filepath          = os.path.join(raw_data_directory, 'yelp_academic_dataset_business.csv')
restaurant_review_filepath = os.path.join(interim_data_directory, 'restaurant_review.csv')
restaurant_filepath        = os.path.join(interim_data_directory, 'restaurant.csv')

In [10]:
# Find .env
dotenv_path = find_dotenv()

# Load entries as environment variables
load_dotenv(dotenv_path)

public_ip = os.environ.get("PUBLIC_IP")
username  = os.environ.get("USERNAME")
password  = os.environ.get("PASSWORD")
port      = os.environ.get("PORT")
database  = os.environ.get("DATABASE")

# Construct database URL from environment variables
uri = f'postgresql://{username}:{password}@{public_ip}:{port}/{database}'
        
# Connection to Postgres database
engine = create_engine(uri)

#### Create table

In [12]:
sql = '''
CREATE TABLE IF NOT EXISTS reviews (
    date          date, 
    stars         integer NOT NULL,
    text          varchar(5000), 
    review_id     varchar(22),
    business_id   varchar(22),
    business_name varchar(64)  
);
'''
# pd.read_sql(sql, con = engine)

#### Load data to database

In [13]:
sql = f'''
COPY reviews(date, stars, text, review_id, business_id, business_name)
FROM '{restaurant_review_filepath}' DELIMITER ',' CSV HEADER;
'''
# pd.read_sql(sql, con = engine)